In [3]:
import math
import cv2
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
# import functions from the scipy.signal module for signal processing
from scipy.signal import butter, filtfilt, find_peaks
import scipy.signal as signal
from scipy.fft import fft, fftfreq


In [ ]:

# Carga el clasificador Haarcascade preentrenado para detección de caras
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Carga el video desde tu computadora, asegurate de que el video esté en la misma carpeta que este archivo y cambia el nombre del video para el archivo que quieras analizar
video_filename = "Juan60.mp4"
# Abre el video
cap = cv2.VideoCapture(video_filename)
fps = cap.get(cv2.CAP_PROP_FPS)
fs = round(fps)
print(fs)

In [ ]:
if not cap.isOpened():
    print("Error al abrir el archivo de video")
else:
    # Ir al fotograma número 30
    frame_num = 30 - 1
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

    # Lee el fotograma
    ret, frame = cap.read()

    if not ret:
        print("Error al leer el fotograma del video")
    else:
        # Convierte la imagen de BGR a RGB para matplotlib
        bordered_frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Mostrar la imagen con matplotlib
        plt.imshow(bordered_frame_rgb)
        plt.title('Original Frame')  # Añadir el título
        plt.show()


In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Ir al fotograma número 30
frame_num = 30 - 1  # Los índices de los fotogramas comienzan en 0

# Establece la posición actual del archivo de video en el fotograma número 30
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

# Lee el fotograma
ret, frame = cap.read()

# Comprueba si el fotograma se ha leído correctamente
if not ret:
    print("Error al leer el fotograma del video")
else:
    # Convertir a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar los rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Dibuja un rectángulo alrededor de cada rostro detectado
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)

    # Convierte la imagen de BGR a RGB para matplotlib
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mostrar la imagen con matplotlib para agregar el título
    plt.imshow(frame_rgb)
    plt.title('Original Frame with Face Detection')  # Añadir el título
    plt.show()

# Libera el capturador de video
#cap.release()

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Ir al fotograma número 30
frame_num = 30 - 1  # Los índices de los fotogramas comienzan en 0

# Establece la posición actual del archivo de video en el fotograma número 30
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

# Lee el fotograma
ret, frame = cap.read()

# Comprueba si el fotograma se ha leído correctamente
if not ret:
    print("Error al leer el fotograma del video")
else:
    # Convertir a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar los rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Asumimos que hay al menos un rostro y tomamos el primero para recortar
    if len(faces) > 0:
        # Coordenadas del rostro detectado
        x, y, w, h = faces[0]

        # Recorta el rostro del fotograma
        face_crop = frame[y:y+h, x:x+w]

        # Convierte la imagen de BGR a RGB para matplotlib
        face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)

        # Mostrar la imagen recortada con matplotlib
        plt.imshow(face_crop_rgb)
        plt.title('Cropped Face')  # Añadir el título
        plt.show()
    else:
        print("No se encontraron rostros en el fotograma.")

In [ ]:
blue_channel, green_channel, red_channel = cv2.split(face_crop)

# Crear una versión del rostro con el canal verde resaltado
# Normalizar los canales al rango 0-1 para evitar la saturación
green_channel_normalized = green_channel / 255.0
red_channel_normalized = red_channel / 255.0
blue_channel_normalized = blue_channel / 255.0

# Resaltar el canal verde incrementándolo y decrementando los otros canales
green_highlighted = cv2.merge([
    (blue_channel_normalized * 0.5).astype(np.float32),
    (green_channel_normalized * 1.5).astype(np.float32),  # Incrementar el verde
    (red_channel_normalized * 0.5).astype(np.float32)
])

# Asegurar que los valores estén dentro del rango 0-1 después del procesamiento
green_highlighted = np.clip(green_highlighted, 0, 1)

# Convertir de vuelta al rango 0-255 para la visualización
green_highlighted = (green_highlighted * 255).astype(np.uint8)

# Mostrar el rostro con el canal verde resaltado
plt.imshow(cv2.cvtColor(green_highlighted, cv2.COLOR_BGR2RGB))
plt.title('Green Channel Face')
plt.show()

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Inicializar lista para almacenar la intensidad promedio del canal verde del rostro en cada fotograma
green_values = []

# Establecer el número de fotogramas a analizar (puedes ajustar este valor)
num_frames_to_analyze = 300  # Por ejemplo, analizar 300 fotogramas

# Leer el video fotograma a fotograma
for _ in range(num_frames_to_analyze):
    ret, frame = cap.read()
    if not ret:
        break  # Si no quedan fotogramas, sal del bucle

    # Convertir a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Extraer la región del rostro
        face_region = frame[y:y+h, x:x+w]

        # Calcular la intensidad promedio del canal verde en la región del rostro
        green_channel = face_region[:, :, 1]
        green_value = np.mean(green_channel)
        green_values.append(green_value)

# Convertir la lista de valores verdes a un array de numpy
green_array = np.array(green_values)

# Mostrar la señal del canal verde
plt.figure(figsize=(10, 4))
plt.plot(green_array, color='green')
plt.title('Green Channel Intensity Over Time in the Face Region')
plt.xlabel('Frame number')
plt.ylabel('Average Green Intensity')
plt.show()


In [ ]:

# Función para crear un filtro pasa banda
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Función para aplicar el filtro pasa banda
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Lista para almacenar los valores medios del canal verde
green_values = []

# Establecer la frecuencia de muestreo (fps del video)
fs = cap.get(cv2.CAP_PROP_FPS)

# Leer el video fotograma a fotograma
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) > 0:
        x, y, w, h = faces[0]
        face_region = frame[y:y+h, x:x+w]
        green_channel = face_region[:, :, 1]
        green_value = np.mean(green_channel)
        green_values.append(green_value)



In [ ]:
# Convertir la lista de valores verdes a un array de numpy
green_array = np.array(green_values)

# Filtrar la señal para eliminar el ruido y enfocarse en las frecuencias del pulso cardíaco
lowcut = 1.0  # Frecuencia mínima del pulso (en Hz)
highcut = 1.6  # Frecuencia máxima del pulso (en Hz)
filtered_green = butter_bandpass_filter(green_array, lowcut, highcut, fs, order=5)

# Realizar la FFT en la señal filtrada
green_fft = fft(filtered_green)
freqs = fftfreq(len(filtered_green), 1/fs)

# Encontrar la frecuencia con la magnitud más alta en el rango de la frecuencia cardíaca
idx = np.argmax(np.abs(green_fft))
pulse_freq = freqs[idx]
heart_rate = pulse_freq * 60  # Convertir a latidos por minuto

# Mostrar el resultado
print(f'Estimated Heart Rate: {heart_rate:.1f} beats per minute')

# Mostrar la señal temporal filtrada y su FFT
plt.figure(figsize=(12, 6))

# Señal temporal filtrada
plt.subplot(2, 1, 1)
plt.plot(filtered_green, color='green')
plt.title('Filtered Green Channel Signal')

# FFT de la señal
plt.subplot(2, 1, 2)
plt.stem(freqs, np.abs(green_fft), 'b', markerfmt=" ", basefmt="-b")
plt.title('Spectral Analysis')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim(0, highcut * 2)  # Limitar el eje x para una mejor visualización
plt.show()


In [ ]:
# Realizar la FFT en la señal filtrada
green_fft = fft(filtered_green)
freqs = fftfreq(len(filtered_green), 1/fs)

# Encontrar la frecuencia con la magnitud más alta en el rango de la frecuencia cardíaca
idx = np.argmax(np.abs(green_fft))
pulse_freq = freqs[idx]
heart_rate = pulse_freq * 60  # Convertir a latidos por minuto

# Mostrar el resultado
print(f'Estimated Heart Rate: {heart_rate:.1f} beats per minute')

# Mostrar la señal temporal filtrada y su FFT
plt.figure(figsize=(12, 6))

# Señal temporal filtrada
plt.subplot(2, 1, 1)
plt.plot(filtered_green, color='green')
plt.title('Filtered Green Channel Signal')

# FFT de la señal
plt.subplot(2, 1, 2)
plt.stem(freqs, np.abs(green_fft), 'b', markerfmt=" ", basefmt="-b")
plt.scatter(pulse_freq, np.abs(green_fft[idx]), color='red')  # Punto para la frecuencia dominante

# Ajustar la posición del texto
# Aquí puedes ajustar las coordenadas para mover el texto
text_x = pulse_freq + 0.1  # Ajusta este valor según sea necesario
text_y = np.abs(green_fft[idx]) - 50  # Ajusta este valor según sea necesario
plt.text(text_x, text_y, f'{pulse_freq:.2f} Hz', color='red')

plt.title('Spectral Analysis')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim(0, highcut * 2)  # Limitar el eje x para una mejor visualización
plt.show()

In [ ]:
# Ahora, graficamos la intensidad del canal verde
plt.figure(figsize=(12, 4))
plt.plot(green_array, color='green')
plt.title('Green Channel Intensity Over Time')
plt.xlabel('Frame number')
plt.ylabel('Average Green Intensity')
plt.show()

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Lista para almacenar los valores medios del canal verde
green_values = []

# Procesar solo los primeros 30 fotogramas
for i in range(30):
    # Leer el fotograma
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma del video")
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Verificar si se encontró al menos un rostro
    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Extraer la región del rostro
        face_region = frame[y:y+h, x:x+w]

        # Extraer el canal verde de la región del rostro
        green_channel = face_region[:, :, 1]

        # Normalizar el canal verde para mejorar la visualización
        green_channel_normalized = cv2.normalize(green_channel, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # Añadir el valor medio del canal verde normalizado a la lista
        green_values.append(np.mean(green_channel_normalized))

        # Crear una imagen en escala de grises con el canal verde como intensidad
        green_visualization = np.stack((green_channel_normalized,) * 3, axis=-1)

        # Visualizar el canal verde en el rostro detectado
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(green_visualization, cv2.COLOR_BGR2RGB))
        plt.title(f'Green Channel in Face Region - Frame {i+1}')
        plt.axis('off')  # Ocultar los ejes
        plt.show()
    else:
        print("No se encontró ningún rostro en el fotograma.")

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Procesar solo los primeros 30 fotogramas
for i in range(30):
    # Leer el fotograma
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma del video")
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Verificar si se encontró al menos un rostro
    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Extraer la región del rostro
        face_region = frame[y:y+h, x:x+w]

        # Extraer el canal verde de la región del rostro
        green_channel = face_region[:, :, 1]
        print(green_channel.shape)
        # Escalar los valores del canal verde para maximizar la visibilidad
        max_green = green_channel.max()
        if max_green > 0:  # Evitar la división por cero
            scale_factor = 255 / max_green
            green_channel_scaled = (green_channel * scale_factor).astype(np.uint8)
        else:
            green_channel_scaled = green_channel

        # Crear una imagen en escala de grises con el canal verde maximizado como intensidad
        green_maximized_visualization = np.zeros_like(face_region)
        green_maximized_visualization[:, :, 1] = green_channel_scaled

        # Visualizar el canal verde maximizado en el rostro detectado
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(green_maximized_visualization, cv2.COLOR_BGR2RGB))
        plt.title(f'Green Channel Maximized in Face Region - Frame {i+1}')
        plt.axis('off')  # Ocultar los ejes
        plt.show()
    else:
        print("No se encontró ningún rostro en el fotograma.")

In [ ]:
cap.release()

In [ ]:

# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Procesar solo los primeros 30 fotogramas
for i in range(30):
    # Leer el fotograma
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma del video")
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Verificar si se encontró al menos un rostro
    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Extraer la región del rostro
        face_region = frame[y:y+h, x:x+w]

        # Extraer el canal verde de la región del rostro
        green_channel = face_region[:, :, 1]

        # Normalizar el canal verde para mejorar la visualización
        green_channel_normalized = cv2.normalize(green_channel, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Crear una imagen en escala de grises con el canal verde como intensidad
        green_visualization = np.zeros_like(face_region)
        green_visualization[:, :, 1] = np.uint8(green_channel_normalized * 255)  # Coloca el canal verde en la imagen

        # Visualizar el canal verde en el rostro detectado
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(green_visualization, cv2.COLOR_BGR2RGB))
        plt.title(f'Green Channel in Face Region - Frame {i+1}')
        plt.axis('off')  # Ocultar los ejes
        plt.show()
    else:
        print("No se encontró ningún rostro en el fotograma.")

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Procesar solo los primeros 30 fotogramas
for i in range(30):
    # Leer el fotograma
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma del video")
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Verificar si se encontró al menos un rostro
    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Definir una ROI para la mejilla (ajustar según sea necesario)
        # Estos valores son estimaciones y podrían necesitar ajuste fino
        mejilla_x = x + int(w * 0.3)  # Comienza un poco a la derecha del lado izquierdo del rostro
        mejilla_y = y + int(h * 0.4)  # Comienza un poco debajo de la mitad del alto del rostro
        mejilla_w = int(w * 0.4)  # Ancho de la mejilla
        mejilla_h = int(h * 0.2)  # Alto de la mejilla

        # Extraer la región de la mejilla
        mejilla_region = frame[mejilla_y:mejilla_y+mejilla_h, mejilla_x:mejilla_x+mejilla_w]

        # Extraer el canal verde de la región de la mejilla
        green_channel = mejilla_region[:, :, 1]

        # Normalizar el canal verde para mejorar la visualización
        green_channel_normalized = cv2.normalize(green_channel, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Visualizar el canal verde en la región de la mejilla
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(mejilla_region, cv2.COLOR_BGR2RGB))
        plt.title(f'Green Channel in Cheek Region - Frame {i+1}')
        plt.axis('off')  # Ocultar los ejes
        plt.show()
    else:
        print("No se encontró ningún rostro en el fotograma.")

In [ ]:
# Asegúrate de que el video se ha abierto correctamente
if not cap.isOpened():
    print("Error al abrir el archivo de video")

# Procesar solo los primeros 30 fotogramas
for i in range(30):
    # Leer el fotograma
    ret, frame = cap.read()
    if not ret:
        print("Error al leer el fotograma del video")
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Verificar si se encontró al menos un rostro
    if len(faces) > 0:
        # Tomar el primer rostro detectado
        x, y, w, h = faces[0]

        # Ajustar la ubicación de la ROI para enfocarse en la mejilla
        mejilla_x = x + int(w * 0.6)  # Ajustar hacia el lado derecho del rostro
        mejilla_y = y + int(h * 0.4)  # Ajustar ligeramente hacia abajo
        mejilla_w = int(w * 0.3)  # Ancho más estrecho
        mejilla_h = int(h * 0.3)  # Alto

        # Extraer la región de la mejilla
        mejilla_region = frame[mejilla_y:mejilla_y+mejilla_h, mejilla_x:mejilla_x+mejilla_w]

        # Extraer el canal verde de la región de la mejilla y normalizarlo
        green_channel = mejilla_region[:, :, 1]
        green_channel_normalized = cv2.normalize(green_channel, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        # Crear una imagen en escala de grises con el canal verde como intensidad
        green_visualization = np.zeros_like(mejilla_region)
        green_visualization[:, :, 1] = np.uint8(green_channel_normalized * 255)  # Coloca el canal verde en la imagen

        # Visualizar el canal verde en la región de la mejilla
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(green_visualization, cv2.COLOR_BGR2RGB))
        plt.title(f'Green Channel in Cheek Region - Frame {i+1}')
        plt.axis('off')  # Ocultar los ejes
        plt.show()
    else:
        print("No se encontró ningún rostro en el fotograma.")